In [52]:
import pandas as pd

data = {
    'Spelling': [],
    'Level': []
}

spelling_levels = {
    '가르치다 vs 가리키다': 2,
    '거름 vs 걸음': 3,
    '껍데기 vs 껍질': 1,
    '낳다 vs 낫다': 2,
    '닫히다 vs 다치다': 4,
    '들리다 vs 들르다': 3,
    '떠올리다 vs 떠오르다': 5,
    '맞히다 vs 맞추다': 4,
    '바라다 vs 바래다': 5,
    '시각 vs 시간': 3,
    '알갱이 vs 알맹이': 2,
    '잃어버리다 vs 잊어버리다': 6,
    '작다 vs 적다': 3,
    '장사 vs 장수': 1,
    '장이 vs 쟁이': 4,
    '적용하다 vs 적응하다': 6,
    '좇다 vs 쫓다': 5,
    '주리다 vs 줄이다': 4,
    '틀리다 vs 다르다': 3,
    '한참 vs 한창': 5,
    '가치 vs 같이': 4,
    '갔다 vs 같다': 2,
    '거치다 vs 걷히다': 3,
    '금새 vs 금세': 2,
    '깁다 vs 깊다': 1,
    '느리다 vs 늘이다': 4,
    '맞다 vs 맡다': 3,
    '매다 vs 메다': 2,
    '바치다 vs 받치다': 5,
    '배다 vs 베다': 6,
    '속다 vs 솎다': 1,
    '시키다 vs 식히다': 3,
    '어떡해 vs 어떻게': 2,
    '이따가 vs 있다가': 4,
    '조리다 vs 졸이다': 5,
    '깨끗이 vs 깨끗히': 6,
    '꼽다 vs 꽂다': 3,
    '담다 vs 담그다': 4,
    '몇 일 vs 며칠': 1,
    '반드시 vs 반듯이': 6,
    '봉오리 vs 봉우리': 2,
    '부치다 vs 붙이다': 4,
    '설거지 vs 설겆이': 3,
    '새배 vs 세배': 2,
    '숫가락 vs 숟가락': 3,
    '왠지 vs 웬지': 5,
    '이 vs 이빨': 1,
    '이파리 vs 잎파리': 2,
    '채 vs 째': 5,
    '케익 vs 케이크': 4
}

for spelling, level in spelling_levels.items():
    data['Spelling'].append(spelling)
    data['Level'].append(level)

data = pd.DataFrame(data)
data[['front', 'back']] = data['Spelling'].str.split(' vs ', expand=True)

In [53]:
import openai

OPEN_API_KEY = ''

openai.api_key = OPEN_API_KEY

def get_completion(spelling, pointer):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "[역할] 당신은 초등학교 국어 선생님입니다. 해당 지문을 확인한 후 단답형 1문제를 출제하세요."},
                {"role": "user", "content": f"[방법]"
                                            f" 아래 주어진 {spelling} 중에 {pointer}단어가 답이 되도록 문제를 만들어줘. 단답형 문제 1개를 출제하세요. 반드시 아래 Output의 유형에 맞는 구성으로 만드세요. 요구한 결과물만 출력해줘"
                                            f"[Input] "
                                            f"[맞춤법]  "
                                            f"{spelling} "
                                            f""
                                            f"[Output]"
                                            f"{example}"}
            ],
            model="gpt-4o-mini"
        )
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

example = """
{
    "question" : "다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요",
    "text" : "밭에서 식물의 성장을 돕기 위해 _____을 주었다.",
    "answer" : "거름",
    "reason" : "식물의 성장을 위해 주는 것은 '거름'이므로 정답은 '거름'입니다."
}
"""

responses = []
level_mapping = {
    1: "미취학아동",
    2: "초등학교 1학년",
    3: "초등학교 2학년",
    4: "초등학교 3학년",
    5: "초등학교 4학년",
    6: "초등학교 5학년",
    7: "초등학교 6학년"
}

for i, row in data.iterrows():
    for x in range (1,7):
        if ((i+1)*6+x)%50==0:
            print((i+1)*6+x)
        response = get_completion(row['Spelling'], row['front'])
        
        if response:
            # print(response)
            responses.append((response, row['Level'], row['Spelling']))
        else:
            responses.append("Failed to get a response from the API.")
    for x in range (1,7):
        if ((i+1)*6+x)%50==0:
            print((i+1)*6+x)
        response = get_completion(row['Spelling'], row['back'])
        
        if response:
            # print(response)
            responses.append((response, row['Level'], row['Spelling']))
        else:
            responses.append("Failed to get a response from the API.")
        
# print(responses)

50
50
100
100
150
150
200
200
250
250
300
300


In [54]:
responses_df = pd.DataFrame(responses, columns=['response', 'level', 'spelling'])

responses_df.to_csv("../data/processed/dt01_21.csv", index=False, encoding='utf-8-sig')

In [55]:
#불러오기
import pandas as pd
import json
from datetime import datetime

responses_df = pd.read_csv("../data/processed/dt01_21.csv", sep=',')

print(responses_df['response'].head())
transformed_data = []

def transform_row(row):
    response_json = json.loads(row['response'])
            
    now = datetime.now().isoformat()
    transformed = {
        "title": response_json.get("question", ""),
        "content": response_json.get("text", ""),
        "answer" : response_json.get("answer", ""),
        "solution": response_json.get("reason", ""),
        "questionTypeCode": "QT02",
        "problemLevelCode": f"PL0{row['level']}",
        "problemTypeCode": "PT01",
        "problemTypeDetailCode": "PT0121",
        "metadata": {
            "options": row['spelling'] 
        },
        "createdDate": now,
        "modifiedDate": now
    }
    
    return transformed


for i, row in responses_df.iterrows():
    try:
        json.loads(row['response'])  # Try parsing the original JSON
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
        print(f"Problematic JSON: {row['response']}")

    
dt01_21_df = pd.DataFrame(transformed_data)

0    {\n    "question" : "다음 문장에서 문맥상 빈칸에 들어갈 적절한 단...
1    {\n    "question" : "다음 문장에서 문맥상 빈칸에 들어갈 적절한 단...
2    {\n    "question": "다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어...
3    {\n    "question" : "다음 문장에서 문맥상 빈칸에 들어갈 적절한 단...
4    {\n    "question" : "다음 문장에서 문맥상 빈칸에 들어갈 적절한 단...
Name: response, dtype: object


In [56]:
print(dt01_21_df)
with open("../data/processed/problem_data/pt0121.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt01_21_df.to_csv("../data/processed/problem_data/pt0121.csv", index=False, encoding='utf-8-sig')

                                title  \
0    다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
1    다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
2    다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
3    다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
4    다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
..                                ...   
595  다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
596  다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
597  다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
598  다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   
599  다음 문장에서 문맥상 빈칸에 들어갈 적절한 단어를 적으세요   

                                         content answer  \
0                  선생님이 학생들에게 새로운 개념을 _____ 주셨다.   가르치다   
1                  선생님은 학생들에게 중요한 내용을 _____ 주셨다.   가르치다   
2                   선생님이 학생에게 새로운 내용을 _____ 주었다.   가르치다   
3                  선생님은 학생들에게 새로운 개념을 _____ 주셨다.   가르치다   
4                  선생님이 학생들에게 새로운 내용을 _____ 주셨다.   가르치다   
..                                           ...    ...   
595      생일 파티를 위해 엄마가 특별하게 만든 _____이 달콤하고 부드럽다.    케이크   
596             내 생일에 친구들이 모여서 ____

In [9]:
import pandas as pd
import json

dt01_21_df = pd.read_csv("../data/processed/problem_data/pt0121.csv", sep=',', encoding='cp949')

# DataFrame을 JSON 형식으로 저장
dt01_21_df.to_json("../data/processed/problem_data/pt0121.json", orient='records', force_ascii=False, indent=4)

# CSV 파일로 다시 저장
dt01_21_df.to_csv("../data/processed/problem_data/pt0121.csv", index=False, encoding='utf-8-sig')